### Script de definição dos limites de DEC e FEC
* Autor: Leonardo da Silveira
* Versão 2.0

##### Conexão com o Google Drive

In [31]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    caminho = '/content/drive/MyDrive/Código e simulações - Python/'
except:
    caminho = ''

##### Importação das Bibliotecas

In [32]:
import numpy as np
import pandas as pd
import scipy.stats as stats

##### Definição das variáveis iniciais

In [33]:
qtdConjSemDesej = 100  # Nº desejável de conj semelhantes
qtdConjSemMin = 50     # Nº mínimo de conj semelhantes
hetMax = 20            # Heterogeneidade máxima [%];

p1 = 0.2     # Percentil base SIN/aéreo
p2 = 0.5     # Percentil base isolado/subt

## Conjuntos a serem processados
distInteresse = '' #Distribuidora de interesse

## Atributos utilizados para DEC e FEC
atrDec = ['PC_VRAM','PLUV','PC_ERMT_3F','PC_NUC_AD','CM_NUC_RES','NUC_IND','NUG'] #,'POT_INST','NUG'
atrFec = ['PC_VRAM','PLUV','PC_ERMT_3F','PC_NUC_AD','CM_NUC_RES','NUC_COM','NUG']

## Atributos cuja complexidade aumenta conforme se eleva o valor do atributo
atrAniPos = ['PC_VRAM','PLUV']

## Colunas contendo os valores de DEC e FEC apurados
decApurado = ['DEC_Ano1','DEC_Ano2','DEC_Ano3']
fecApurado = ['FEC_Ano1','FEC_Ano2','FEC_Ano3']

atributos = list(set(atrDec + atrFec)) # Lista de atributos
# atributos = ['PC_VRAM','PLUV','PC_ERMT_3F','PC_NUC_AD','CM_NUC_RES','NUC_IND','NUC_COM','POT_INST','NUG'] # Lista de atributos
qtdAtrDec = len(atrDec) # Quantidade de atributos para DEC
qtdAtrFec = len(atrFec) # Quantidade de atributos para FEC

## Sufixo do nome dos arquivos de saída
sufixo = '_NUG_NUC_2019' #Ex.: '_caso1'

##### Importação dos dados de entrada

In [34]:
data = pd.read_excel(caminho + '02_Dados em Processamento/Atributos_GD_NUC_2019.xlsx')
data.set_index('Cod_Conj', inplace=True)
data.drop(columns=['DistribuidoraConjunto','Regiao','CcMed_Ano1','CcMed_Ano2','CcMed_Ano3'], inplace=True)

##### Remover conjuntos da clusterização

In [35]:
# conjuntosRemovidos = [13002,15799,13004,15800,13006,13007,15798,13008,13009,13013,13014,16103,16104,16105,16106,16107,16108,12727,12731,12729,16109,16110,16111,16112,16113,16114,16115,16116,13100,13101,13102,13103,13104,15723,13105,13106,13107,13108,13111,13112,13114,13113,13115,13116,13117,13118,13119,13120,13121,13122,13123,13124,13125,13126,13127,13128,13129,13130,13131,13132,13133,13134,15724,13136,13137,13138,14011,14012,14013,14014,15640,14016,14017,14018,14019,14020,14021,14022,14023,14024,14025,14026,14027,15642,14029,15645,14030,14031,15643,15641,14032,14033,14034,14035,14036,14037,14038,15644,14040,14041,14042,14043,14044,15812,14045,14046,14047,14048,14049,14050,14051,14052,14053,14054,14055,14056,14057,14058,13195,13206,13227,13196,13197,13198,13200,13203,13210,13211,13212,13213,15787,13222,13223,13224,13225]
# data.drop(index=conjuntosRemovidos, inplace=True)

if data[atributos].isna().values.any():
    data.dropna(subset = atributos, inplace=True)
    print('Foram removidos conjuntos com atributos faltantes.')

Foram removidos conjuntos com atributos faltantes.


##### DEC e FEC médios

In [36]:
data['DEC'] = data[decApurado].replace(0, np.nan).mean(axis=1, skipna = True)
data['FEC'] = data[fecApurado].replace(0, np.nan).mean(axis=1, skipna = True)
data['DEC_FEC'] = data['DEC']/data['FEC']

##### Atributo Normalizado Individual

In [37]:
medAtr = data[atributos].mean(axis=0, skipna = True)
desAtr = data[atributos].std(axis=0, skipna = True)
maxAtr = data[atributos].where(data[atributos] < medAtr + 3 * desAtr, np.nan).max(axis=0, skipna = True)
minAtr = data[atributos].min(axis=0, skipna = True)
ani = 100-100*(data[atributos]-minAtr)/(maxAtr-minAtr)
ani[atrAniPos] = 100-ani[atrAniPos]

##### Conjuntos aéreos/subterrâneos

In [38]:
data['Subt'] = data['Padrao'].apply(lambda x: True if x==2 or x==3 else False)

##### Normalização z-score

In [39]:
atribNorm = stats.zscore(data[atributos], axis=0, ddof=1)

##### Conjuntos a serem processados

In [40]:
dataInteresse = data[data['Distribuidora']==distInteresse] if distInteresse else data

##### Definição dos conjuntos semelhantes DEC

In [41]:
resultadoDec = pd.DataFrame()
conjSemDec = pd.DataFrame()

for index, row in dataInteresse.iterrows():
  
  a = (atribNorm[atrDec]-atribNorm[atrDec].loc[index]).pow(2).sum(axis=1).pow(1./2)*100/(3 * np.sqrt(qtdAtrDec))
  a = a.sort_values().to_frame(name='HeterogeneidadeDEC')
  a['CodConjInteresse']=index
  a['ConjInteresse']=row['Conjunto']
  a['CodConjSemeDEC']=a.index

  a = a.join(data[['Conjunto','Distribuidora','Padrao','Localizacao','Subt','DEC']])
  a.drop(a[a['Subt'] != row['Subt']].index, inplace=True)
  a = a[:qtdConjSemDesej]
  while (a['HeterogeneidadeDEC'].iloc[-1] > hetMax) and len(a.index)>qtdConjSemMin:
    a = a[:-1]
  conjSemDec = pd.concat([conjSemDec,a[['CodConjInteresse','ConjInteresse','HeterogeneidadeDEC','CodConjSemeDEC']]], 
                         ignore_index=True, sort=False)

  hmg = a['HeterogeneidadeDEC'].iloc[-1]
  aniMedio = ani[atrDec].loc[a.index].mean(axis=0, skipna = True)
  scoreAni = (ani[atrDec].loc[index] - aniMedio).sum()/qtdAtrDec

  percentil = p2 if row['Subt'] or row['Localizacao'] == 2 else p1

  if hmg > hetMax:
    if       scoreAni < -3 : percentil-=0.1
    if -3 <= scoreAni <  3 : percentil+=0
    if  3 <= scoreAni <  6 : percentil+=0.1
    if  6 <= scoreAni <  9 : percentil+=0.2
    if  9 <= scoreAni      : percentil+=0.3

  qtdConjSem = len(a.index)
  idxConjRef = np.floor((qtdConjSem - 1) * percentil + 1).astype(int)
  conjRef = a.sort_values(by=['DEC']).iloc[[idxConjRef-1]]
  limObj = np.ceil(conjRef['DEC'].iloc[0])

  d = {
      'ConjuntosSemelhantesDEC': qtdConjSem,
      'HeterogeneidadeDEC': hmg,
      'Score_ANI_DEC': scoreAni,
      # 'Percentil': 'P' + str(int(100*p)),
      'Conjunto_ReferênciaDEC_ANI': conjRef['Conjunto'].iloc[0],
      'Cod_Conj_Ref_DEC': conjRef.index,
      # 'DEC_Conj_Ref_DEC': conjRef['DEC'].iloc[0],
      'DEC_Lim_Obj': limObj,
      }
  resultadoDec = pd.concat([resultadoDec, pd.DataFrame(data=d, index=[index])])
resultadoDec

,ConjuntosSemelhantesDEC,HeterogeneidadeDEC,Score_ANI_DEC,Conjunto_ReferênciaDEC_ANI,Cod_Conj_Ref_DEC,DEC_Lim_Obj
12584,50,26.588483,2.806319,MUANÁ,14490,26.0
12581,50,26.930753,-1.916530,URUARÁ,15907,40.0
12583,50,29.008598,1.100828,CASTELO DOS SONHOS,14458,26.0
12582,50,23.181183,0.311522,CLAUDIA,14793,37.0
13010,50,20.711860,-0.032257,GURUPI URBANO,16026,6.0
...,...,...,...,...,...,...
15602,100,14.397836,1.432105,Mateus Leme,15281,11.0
15601,100,16.743285,1.062397,Lagoa Santa,16304,13.0
12737,100,17.477506,1.152000,ARARUNA,13685,12.0
12743,100,14.768272,1.908721,Rio do Prado,16325,11.0


##### Definição dos conjuntos semelhantes FEC

In [42]:
resultadoFec = pd.DataFrame()
conjSemFec = pd.DataFrame()

for index, row in dataInteresse.iterrows():
  
  a = (atribNorm[atrFec]-atribNorm[atrFec].loc[index]).pow(2).sum(axis=1).pow(1./2)*100/(3 * np.sqrt(qtdAtrFec))
  a = a.sort_values().to_frame(name='HeterogeneidadeFEC')
  a['CodConjInteresse']=index
  a['ConjInteresse']=row['Conjunto']
  a['CodConjSemeFEC']=a.index

  a = a.join(data[['Conjunto','Distribuidora','Padrao','Localizacao','Subt','FEC']])
  a.drop(a[a['Subt'] != row['Subt']].index, inplace=True)
  a = a[:qtdConjSemDesej]
  while (a['HeterogeneidadeFEC'].iloc[-1] > hetMax) and len(a.index)>qtdConjSemMin:
    a = a[:-1]
  conjSemFec = pd.concat([conjSemFec,a[['CodConjInteresse','ConjInteresse','HeterogeneidadeFEC','CodConjSemeFEC']]], 
                         ignore_index=True, sort=False)

  hmg = a['HeterogeneidadeFEC'].iloc[-1]
  aniMedio = ani[atrFec].loc[a.index].mean(axis=0, skipna = True)
  scoreAni = (ani[atrFec].loc[index] - aniMedio).sum()/qtdAtrFec

  percentil = p2 if row['Subt'] or row['Localizacao'] == 2 else p1

  if hmg > hetMax:
    if       scoreAni < -3 : percentil-=0.1
    if -3 <= scoreAni <  3 : percentil+=0
    if  3 <= scoreAni <  6 : percentil+=0.1
    if  6 <= scoreAni <  9 : percentil+=0.2
    if  9 <= scoreAni      : percentil+=0.3

  qtdConjSem = len(a.index)
  idxConjRef = np.floor((qtdConjSem - 1) * percentil + 1).astype(int)
  conjRef = a.sort_values(by=['FEC']).iloc[[idxConjRef-1]]
  limObj = np.ceil(conjRef['FEC'].iloc[0])

  d = {
      'ConjuntosSemelhantesFEC': qtdConjSem,
      'HeterogeneidadeFEC': hmg,
      'Score_ANI_FEC': scoreAni,
      # 'Percentil': 'P' + str(int(100*p)),
      'Conjunto_ReferênciaFEC_ANI': conjRef['Conjunto'].iloc[0],
      'Cod_Conj_Ref_FEC': conjRef.index,
      # 'FEC_Conj_Ref_FEC': conjRef['DEC'].iloc[0],
      'FEC_Lim_Obj': limObj,
      }
  resultadoFec = pd.concat([resultadoFec, pd.DataFrame(data=d, index=[index])])
resultadoFec

,ConjuntosSemelhantesFEC,HeterogeneidadeFEC,Score_ANI_FEC,Conjunto_ReferênciaFEC_ANI,Cod_Conj_Ref_FEC,FEC_Lim_Obj
12584,50,28.351573,0.938014,ALTAMIRA I,15461,15.0
12581,50,58.150643,7.348605,Ji-Parana,12650,12.0
12583,50,40.595711,0.631289,ORIXIMINÁ,14496,14.0
12582,50,40.308880,0.415870,COMODORO,14795,15.0
13010,50,22.225463,0.323047,JOINVILLE IV,16049,4.0
...,...,...,...,...,...,...
15602,100,16.820872,-0.143130,Pedro Leopoldo 3,16319,6.0
15601,100,17.026820,1.383589,TEIXEIRA,13742,6.0
12737,100,19.951877,-0.159386,PRINCESA ISABEL,13727,5.0
12743,100,15.959258,1.138901,RIACHÃO,13729,5.0


##### Cálculo da trajetória de limites

In [43]:
trajLimites = pd.DataFrame()

for index, row in dataInteresse.iterrows():
  decLimObj = resultadoDec['DEC_Lim_Obj'].loc[index]
  fecLimObj = resultadoFec['FEC_Lim_Obj'].loc[index]

  if row['DECL_V0'] > decLimObj:
    trajConj = {
        'DEC_V1': np.around(row['DECL_V0'] - 1 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V2': np.around(row['DECL_V0'] - 2 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V3': np.around(row['DECL_V0'] - 3 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V4': np.around(row['DECL_V0'] - 4 * (row['DECL_V0'] - decLimObj) / 8),
        'DEC_V5': np.around(row['DECL_V0'] - 5 * (row['DECL_V0'] - decLimObj) / 8),
    }
  else:
    trajConj = {
        'DEC_V1': row['DECL_V0'],
        'DEC_V2': row['DECL_V0'],
        'DEC_V3': row['DECL_V0'],
        'DEC_V4': row['DECL_V0'],
        'DEC_V5': row['DECL_V0'],
    }
  if row['FECL_V0'] > fecLimObj:
    trajConj.update({
        'FEC_V1': np.around(row['FECL_V0'] - 1 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V2': np.around(row['FECL_V0'] - 2 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V3': np.around(row['FECL_V0'] - 3 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V4': np.around(row['FECL_V0'] - 4 * (row['FECL_V0'] - fecLimObj) / 8),
        'FEC_V5': np.around(row['FECL_V0'] - 5 * (row['FECL_V0'] - fecLimObj) / 8),
    })
  else:
    trajConj.update({
        'FEC_V1': row['FECL_V0'],
        'FEC_V2': row['FECL_V0'],
        'FEC_V3': row['FECL_V0'],
        'FEC_V4': row['FECL_V0'],
        'FEC_V5': row['FECL_V0'],
    })

  trajLimites = pd.concat([trajLimites, pd.DataFrame(data=trajConj, index=[index])])

trajLimites.index.name='Cod_Conj'

##### Resultado

In [44]:
resultado = dataInteresse[['Conjunto','Cod_Dist','Distribuidora']]
resultado = resultado.join(resultadoDec)
resultado = resultado.join(resultadoFec)

## Planilha Conjuntos Semelhantes DEC
semelhantesDEC = conjSemDec.join(data[['Conjunto','Distribuidora']+atrDec], on='CodConjSemeDEC')
semelhantesDEC = semelhantesDEC.rename(columns={'Conjunto': 'ConjSemeDEC', 'Distribuidora': 'DistSemeDEC'})

## Planilha Conjuntos Semelhantes FEC
semelhantesFEC = conjSemFec.join(data[['Conjunto','Distribuidora']+atrFec], on='CodConjSemeFEC')
semelhantesFEC = semelhantesFEC.rename(columns={'Conjunto': 'ConjSemeFEC', 'Distribuidora': 'DistSemeFEC'})

##### Exportação do resultado

In [45]:

trajLimites.to_excel(caminho + '03_Dados de Saída/TRAJETORIA'+ sufixo +'.xlsx', index=True)

# Planilha RESULTADO.xlsx
with pd.ExcelWriter(
    path = caminho + '03_Dados de Saída/RESULTADO'+ sufixo +'.xlsx',
) as writer:
    resultado.to_excel(writer, sheet_name="Resultado", index=True, na_rep='NaN')
    columnLength=[9,37,9,18,25,20,14,37,18,12,25,20,14,37,18,12]
    for idx, val in enumerate(columnLength):
        writer.sheets['Resultado'].set_column(idx, idx, val)

# Planilha CONJUNTOSSEMELHANTES_DEC.xlsx
# with pd.ExcelWriter(
#     path = caminho + '03_Dados de Saída/CONJUNTOSSEMELHANTES_DEC'+ sufixo +'.xlsx',
# ) as writer:
#     semelhantesDEC.to_excel(writer, sheet_name="SemelhantesDEC", index=False, na_rep='NaN')
#     columnLength=[17,37,20,17,37,18,10,9,12,12,13,9]
#     for idx, val in enumerate(columnLength):
#         writer.sheets['SemelhantesDEC'].set_column(idx, idx, val)

# Planilha CONJUNTOSSEMELHANTES_FEC.xlsx
# with pd.ExcelWriter(
#     path = caminho + '03_Dados de Saída/CONJUNTOSSEMELHANTES_FEC'+ sufixo +'.xlsx',
# ) as writer:
#     semelhantesFEC.to_excel(writer, sheet_name="SemelhantesFEC", index=False, na_rep='NaN')
#     columnLength=[17,37,20,17,37,18,10,9,12,12,13,9]
#     for idx, val in enumerate(columnLength):
#         writer.sheets['SemelhantesFEC'].set_column(idx, idx, val)